# LSTM - 2.0

Una seconda implementazione di LSTM senza sovrapposizione delle finestre temporali.

## Caricamento del dataframe

In [ ]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn as nn

import numpy as np
import pandas as pd
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score

from torch.utils.tensorboard import SummaryWriter

from scipy.stats import mode

import os
import shutil
import glob

main_folder = "../data"

folder = "./runs/LSTM_1"

try:
    shutil.rmtree(folder, ignore_errors=True)
except:
    pass

try:
    os.mkdir(folder)
except:
    pass

writer = SummaryWriter(folder)

## Train - Validation - Test

In [ ]:
LABELS = [
    "No_action",
    "Prendi",
    "Rilascia",
    "Premi"
]

In [ ]:
class2idx = {
    "No_action": 0,
    "Prendi": 1,
    "Rilascia": 2,
    "Premi": 3
}

idx2class = {v: k for k, v in class2idx.items()}

In [ ]:
def join_csv(folder_set, filename):
    csv:list = []

    for file in glob.glob(folder_set + "/*.csv"):
        csv.append(file)

    columns:bool = True
    with open(filename, "w") as f:
        for fcsv in csv:
            with open(fcsv, "r") as fc:
                if columns:
                    f.writelines(fc.readlines())
                    columns = False
                fc.readline()
                f.writelines(fc.readlines())

In [ ]:
def create_set(folder_set):
    csv:list = []
    for file in glob.glob(folder_set + "/*.csv"):
        csv.append(file)
    data = []
    target = []
    for fcsv in csv:
        data_video = pd.read_csv(fcsv, usecols = [i for i in range(156)]).to_numpy()
        target_video = pd.read_csv(fcsv, usecols = ["TARGET"])
        target_video["TARGET"].replace(class2idx, inplace=True)
        data.append(data_video)
        target.append(target_video.to_numpy())
    return (np.array(data, dtype=object), np.array(target, dtype=object))

In [ ]:
folder_set = [[main_folder + "/train_set", main_folder + "/train.csv"], [main_folder + "/test_set", main_folder + "/test.csv"], [main_folder + "/val_set", main_folder + "/val.csv"]]

train_array, train_label_array = create_set(folder_set[0][0])
test_array, test_label_array = create_set(folder_set[1][0])
val_array, val_label_array = create_set(folder_set[2][0])

In [ ]:
print(train_array.shape, train_label_array.shape, test_array.shape, test_label_array.shape, val_array.shape, val_label_array.shape)

In [ ]:
for elem, label in zip(train_array, train_label_array):
    print(elem.shape, label.shape)
    print(type(elem), type(label))

In [ ]:
folder_set = [[main_folder + "/train_set", main_folder + "/train.csv"], [main_folder + "/test_set", main_folder + "/test.csv"], [main_folder + "/val_set", main_folder + "/val.csv"]]

for f_set, filename in folder_set:
    join_csv(f_set, filename)
    
train = pd.read_csv(main_folder + "/train.csv")
test = pd.read_csv(main_folder + "/test.csv")
val = pd.read_csv(main_folder + "/val.csv")

train['TARGET'].replace(class2idx, inplace=True)
test['TARGET'].replace(class2idx, inplace=True)
val['TARGET'].replace(class2idx, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
val.head()

In [ ]:
print(train.shape[0], test.shape[0], val.shape[0])

Si separano le colonne delle features dall'etichetta.

In [ ]:
train_label = train["TARGET"]
test_label = test["TARGET"]
val_label = val["TARGET"]

del train["TARGET"]
del test["TARGET"]
del val["TARGET"]

## Visualizzazione della distribuzione delle classi in Train, Val e Test

In [ ]:
def get_class_distribution(obj):
    count_dict = {
        "No_action": 0,
        "Prendi": 0,
        "Rilascia": 0,
        "Premi": 0,
    }
    
    for i in obj:
        if i == 0: 
            count_dict['No_action'] += 1
        elif i == 1: 
            count_dict['Prendi'] += 1
        elif i == 2: 
            count_dict['Rilascia'] += 1
        elif i == 3: 
            count_dict['Premi'] += 1            
        else:
            print("Check classes.")
            
    return count_dict

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(25,7))
# Train
sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution(train_label)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[0]).set_title('Class Distribution in Train Set')
# Validation
sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution(val_label)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[1]).set_title('Class Distribution in Val Set')
# Test
sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution(test_label)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[2]).set_title('Class Distribution in Test Set')

## Weighted Sampling

Poiché c'è uno squilibrio di classe, utilizziamo la suddivisione stratificata per creare i nostri set di train, convalida e test.

In [ ]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
def create_dataset(dataset, target, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        dataX.append(dataset[i:(i + window_size)])
        dataY.append(target[i:(i + window_size + 1)].values[0])
    return np.array(dataX), np.array(dataY)

In [ ]:
train_X, train_Y = create_dataset(train, train_label, 1)
train_dataset_w = ClassifierDataset(torch.from_numpy(train_X).float(), torch.from_numpy(train_Y).long())

In [ ]:
target_list = []

for _, t in train_dataset_w:
    target_list.append(t)
    
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

Quindi, otteniamo il conteggio di tutte le classi nel nostro set di allenamento.  Usiamo il reciproco di ogni conteggio per ottenere il suo peso.  Ora che abbiamo calcolato i pesi per ogni classe, possiamo procedere.

In [ ]:
class_count = [i for i in get_class_distribution(train_label).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 

print(class_weights)

WeightedRandomSampler prevede un peso per ogni campione.  Lo usiamo come segue.

In [ ]:
class_weights_all = class_weights[target_list]

Infine, inizializziamo il nostro WeightedRandomSampler.  Lo chiameremo nel nostro dataloader in seguito.

In [ ]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

## Parametri del modello

In [ ]:
class ClassifierDataset(Dataset):
    
    def __init__(self, array, label, index_data, window):
        self.index_data = index_data
        self.array = array
        self.label = label
        self.window = window
        
    def __getitem__(self, index):
        file_index = self.index_data[index][0].tolist()
        i = self.index_data[index][1]
        j = self.index_data[index][2]
        if i == j:
            x_data = [self.array[file_index][i]] * self.window
            y_data = self.label[file_index][i]
            X = torch.from_numpy(np.array(x_data).astype(float)).float()
            Y = torch.from_numpy(np.array([y_data]).astype(int)).long()
            return X, Y
        elif j-i < self.window-1:
            x_data = [self.array[file_index][i]] * (self.window-j)
            x_data.extend(self.array[file_index][(i+1):(j+1)])
            x_data = np.array(x_data).astype(float)
        else:
            x_data = self.array[file_index][i:(j+1)]
            k = j-1
            while len(x_data) > self.window:
                x_data = self.array[file_index][i:(k+1)]
                k -= 1
        X = torch.from_numpy(x_data).float()
        y_data = self.label[file_index][i:(j+1)]
        y_mode = mode(y_data)[0][0]
        Y = torch.from_numpy(np.array([y_mode])).long()
        return X, Y
        
    def __len__ (self):
        return len(self.index_data)

In [ ]:
def create_dataset(array_data, label_data, window):
    X = []
    files_num = len(array_data)
    for index, array, label in zip(range(files_num), array_data, label_data):
        l = array.shape[0]
        i = 0
        while l - i >= window:
            j = i + window
            index_window = np.array([index, i, j]).astype(int)
            X.append(index_window)
            i += window
        if i < l:
            w = l - i
            i = i - window
            i = i + w
            j = i + window
            index_window = np.array([index, i, j]).astype(int)
            X.append(index_window)
    X_data = torch.from_numpy(np.array(X).astype(int))
    return ClassifierDataset(array_data, label_data, X_data, window)

In [ ]:
def create_dataset2(array_data, label_data, window):
    X = []
    files_num = len(array_data)
    for index, array, label in zip(range(files_num), array_data, label_data):
        l = array.shape[0]
        i = -1
        w = [0] * window
        while i < l:
            i += 1 
            w.pop(0)
            w.append(i)
            index_window = np.array([index, w[0], i]).astype(int)
            X.append(index_window)
    X_data = torch.from_numpy(np.array(X).astype(int))
    return ClassifierDataset(array_data, label_data, X_data, window)

In [ ]:
EPOCHS = 200
window = 3
BATCH_SIZE = 16
LEARNING_RATE = 0.05
NUM_LAYER = 1
NUM_HIDDEN = 192
NUM_FEATURES = 156
NUM_CLASSES = 4
index_name = 51
model_name = "000" + str(index_name) + "_mymodel.pt"

In [ ]:
train_dataset = create_dataset(train_array, train_label_array, window)
test_dataset = create_dataset2(test_array, test_label_array, window)
val_dataset = create_dataset2(val_array, val_label_array, window)

In [ ]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

val_loader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [ ]:
print(train_dataset.index_data.shape)
print(len(train_dataset))
print(test_dataset.index_data.shape)
print(len(test_dataset))
print(val_dataset.index_data.shape)
print(len(val_dataset))

## Rete Neurale LSTM

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, window, output_size, hidden_layer_size, num_layers):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_layer_size
        self.output_size = output_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers)
        self.regressor = nn.Linear(hidden_layer_size, output_size)
        
    def forward(self, x, hidden=None):
        if hidden is not None:
            h0 = hidden[0]
            c0 = hidden [1]
        else:
            h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(device)
        e = x.view(x.size(1), x.size(0), x.size(2))
        h, hn = self.lstm(e, (h0, c0))
        h = h.view(h.size(1), h.size(0), h.size(2))
        h = h[:,-1,:]
        h = self.regressor(h)
        return h, hn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
model = LSTM(input_size=NUM_FEATURES, window=window, output_size=NUM_CLASSES, hidden_layer_size=NUM_HIDDEN, num_layers=NUM_LAYER)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
print(model)

## Train the model

Prima di iniziare il nostro addestramento, definiamo una funzione per calcolare la precisione per epoca.

Questa funzione accetta y_pred e y_test come argomenti di input.  Quindi applichiamo log_softmax a y_pred ed estraiamo la classe che ha una probabilità maggiore.

Successivamente, confrontiamo le classi previste e le classi effettive per calcolare l'accuratezza.

In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

In [ ]:
def accuracy(y_pred, y_true):
    return accuracy_score(y_true, y_pred) * 100

In [ ]:
def f1_s(y_pred, y_true):
    return f1_score(y_true, y_pred, average=None, zero_division=1, labels=[0,1,2,3])

In [ ]:
print('test_data_size:',test_dataset.index_data.size())
print('test_label_size:',test_dataset.index_data.size())
n_data_size_test = test_dataset.index_data.size()[0]
print('n_data_size_test:',n_data_size_test)

print('train_data_size:',train_dataset.index_data.size())
print('train_label_size:',train_dataset.index_data.size())
n_data_size_train = train_dataset.index_data.size()[0]
print('n_data_size_train:',n_data_size_train)

print('val_data_size:',val_dataset.index_data.size())
print('val_label_size:',val_dataset.index_data.size())
n_data_size_val = val_dataset.index_data.size()[0]
print('n_data_size_val:',n_data_size_val)

Definiremo anche 2 dizionari che memorizzeranno l'accuratezza/epoca e la perdita/epoca sia per il treno che per i set di convalida.

In [ ]:
accuracy_stats = {
    'train': [],
    "val": []
}

loss_stats = {
    'train': [],
    "val": []
}

f1_score_No_action = {
    'train': [],
    "val": []
}

f1_score_Prendi = {
    'train': [],
    "val": []
}

f1_score_Rilascia = {
    'train': [],
    "val": []
}

f1_score_Premi = {
    'train': [],
    "val": []
}

In [ ]:
print("Begin training.")

print_every = 500

for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    train_epoch_f1_No_action = 0
    train_epoch_f1_Prendi = 0
    train_epoch_f1_Rilascia = 0
    train_epoch_f1_Premi = 0
    train_pred_lab = []
    train_true_lab = []
    
    model.train()
    
    hidden_train = None
    
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        y_train_batch = torch.squeeze(y_train_batch)
        optimizer.zero_grad()
        
        y_train_pred, _ = model(X_train_batch, hidden_train)
        
        #h0, c0 = hidden_train
        #h0.detach_(), c0.detach_()
        #hidden_train = (h0, c0)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_pred_lab.append(y_train_pred)
        train_true_lab.append(y_train_batch)
        
        train_loss.backward()
        optimizer.step()
                
        train_epoch_loss += train_loss.item()
        
    y_pred_tags = []
    y_true_tags = []
    for (p, t) in zip(train_pred_lab, train_true_lab):
        _, y_tags = torch.max(p, dim = 1)
        y_pred_tags.extend(y_tags.tolist())
        y_true_tags.extend(t.tolist())
    
    train_acc = accuracy(y_pred_tags, y_true_tags)
    train_f1 = f1_s(y_pred_tags, y_true_tags)
    
    train_epoch_acc += train_acc.item()
    train_epoch_f1_No_action += train_f1[0].item()
    train_epoch_f1_Prendi += train_f1[1].item()
    train_epoch_f1_Rilascia += train_f1[2].item()
    train_epoch_f1_Premi += train_f1[3].item()
        
    if e % print_every == 0:
        
        # VALIDATION    
        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_acc = 0
            val_epoch_f1 = 0
            val_epoch_f1_No_action = 0
            val_epoch_f1_Prendi = 0
            val_epoch_f1_Rilascia = 0
            val_epoch_f1_Premi = 0
            val_pred_lab = []
            val_true_lab = []

            #hidden_valid = (h0, c0)
            hidden_valid = None

            model.eval()
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)

                y_val_pred, _ = model(X_val_batch, hidden_valid)

                val_loss = criterion(y_val_pred, y_val_batch[0][0])
                val_pred_lab.append(y_val_pred)
                val_true_lab.append(y_val_batch[0][0])
                
                val_epoch_loss += val_loss.item()
                
            y_pred_tags = []
            y_true_tags = []
            for (p, t) in zip(val_pred_lab, val_true_lab):
                _, y_tags = torch.max(p, dim = 1)
                y_pred_tags.extend(y_tags.tolist())
                y_true_tags.extend(t.tolist())
                
            val_acc = accuracy(y_pred_tags, y_true_tags)
            val_f1 = f1_s(y_pred_tags, y_true_tags)

            val_epoch_acc += val_acc.item()
            val_epoch_f1_No_action += val_f1[0].item()
            val_epoch_f1_Prendi += val_f1[1].item()
            val_epoch_f1_Rilascia += val_f1[2].item()
            val_epoch_f1_Premi += val_f1[3].item()

        loss_stats['train'].append(train_epoch_loss/len(train_loader))        
        loss_stats['val'].append(val_epoch_loss/len(val_loader))
        accuracy_stats['train'].append(train_epoch_acc)
        accuracy_stats['val'].append(val_epoch_acc)
        f1_score_No_action['train'].append(train_epoch_f1_No_action)
        f1_score_No_action['val'].append(val_epoch_f1_No_action)
        f1_score_Prendi['train'].append(train_epoch_f1_Prendi)
        f1_score_Prendi['val'].append(val_epoch_f1_Prendi)
        f1_score_Rilascia['train'].append(train_epoch_f1_Rilascia)
        f1_score_Rilascia['val'].append(val_epoch_f1_Rilascia)
        f1_score_Premi['train'].append(train_epoch_f1_Premi)
        f1_score_Premi['val'].append(val_epoch_f1_Premi)

        writer.add_scalars("Loss/train_val", {
            "train": train_epoch_loss/len(train_loader),
            "val": val_epoch_loss/len(val_loader),
        }, e)
        writer.add_scalars("Acc/train_val", {
            "train": train_epoch_acc,
            "val": val_epoch_acc,
        }, e)
        writer.add_scalars("F1-No_action/train_val", {
            "train": train_epoch_f1_No_action,
            "val": val_epoch_f1_No_action,
        }, e)
        writer.add_scalars("F1-Prendi/train_val", {
            "train": train_epoch_f1_Prendi,
            "val": val_epoch_f1_Prendi,
        }, e)
        writer.add_scalars("F1-Rilascia/train_val", {
            "train": train_epoch_f1_Rilascia,
            "val": val_epoch_f1_Rilascia,
        }, e)
        writer.add_scalars("F1-Premi/train_val", {
            "train": train_epoch_f1_Premi,
            "val": val_epoch_f1_Premi,
        }, e)
        writer.flush()
        print(f'Epoch {e+0:03}:\
              | Train Loss: {train_epoch_loss/len(train_loader):.5f}\
              | Val Loss: {val_epoch_loss/len(val_loader):.5f}\
              | Train Acc: {train_epoch_acc:.3f}\
              | Val Acc: {val_epoch_acc:.3f}\
              | Train F1-No_action: {train_epoch_f1_No_action:.5f}\
              | Val F1-No_action: {val_epoch_f1_No_action:.5f}\
              | Train F1-Prendi: {train_epoch_f1_Prendi:.5f}\
              | Val F1-Prendi: {val_epoch_f1_Prendi:.5f}\
              | Train F1-Rilascia: {train_epoch_f1_Rilascia:.5f}\
              | Val F1-Rilascia: {val_epoch_f1_Rilascia:.5f}\
              | Train F1-Premi: {train_epoch_f1_Premi:.5f}\
              | Val F1-Premi: {val_epoch_f1_Premi:.5f}')
writer.close()
print("Done.")

## Visualizzazione Loss e Accuracy

Per tracciare i grafici delle linee di perdita e accuratezza, creiamo nuovamente un dataframe dai dizionari precision_stats e loss_stats.

In [ ]:
# Create dataframes
train_val_acc_df = pd.DataFrame.from_dict(accuracy_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_loss_df = pd.DataFrame.from_dict(loss_stats).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_f1_No_action = pd.DataFrame.from_dict(f1_score_No_action).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_f1_Prendi = pd.DataFrame.from_dict(f1_score_Prendi).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_f1_Rilascia = pd.DataFrame.from_dict(f1_score_Rilascia).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})
train_val_f1_Premi = pd.DataFrame.from_dict(f1_score_Premi).reset_index().melt(id_vars=['index']).rename(columns={"index":"epochs"})

# Plot the dataframes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,7))

sns.lineplot(data=train_val_acc_df, x = "epochs", y="value", hue="variable", ax=axes[0]).set_title('Train-Val Accuracy/Epoch')
sns.lineplot(data=train_val_loss_df, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val Loss/Epoch')

In [ ]:
# Plot the dataframes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,7))

sns.lineplot(data=train_val_f1_No_action, x = "epochs", y="value", hue="variable", ax=axes[0]).set_title('Train-Val F1-No_action/Epoch')
sns.lineplot(data=train_val_f1_Prendi, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val F1-Prendi/Epoch')

In [ ]:
# Plot the dataframes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,7))

sns.lineplot(data=train_val_f1_Rilascia, x = "epochs", y="value", hue="variable", ax=axes[0]).set_title('Train-Val F1-Rilascia/Epoch')
sns.lineplot(data=train_val_f1_Premi, x = "epochs", y="value", hue="variable", ax=axes[1]).set_title('Train-Val F1-Premi/Epoch')

## Test the model

Al termine dell'addestramento, dobbiamo testare come si è comportato il nostro modello.  Nota che abbiamo usato model.eval() prima di eseguire il nostro codice di test.  Per dire a PyTorch che non vogliamo eseguire la retropropagazione durante l'inferenza, usiamo torch.no_grad(), proprio come abbiamo fatto per il ciclo di convalida sopra.

Iniziamo definendo un elenco che conterrà le nostre previsioni.  Quindi eseguiamo il ciclo dei nostri batch utilizzando test_loader.

In [ ]:
y_pred_list = []
y_true_list = []

with torch.no_grad():
    model.eval()
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred, _ = model(X_batch)
        guess, guess_i = categoryFromOutput(y_test_pred)
        y_pred_list.append(guess_i)
        y_true_list.append(y_batch[0][0].tolist()[0])

In [ ]:
len(y_pred_list) == len(y_true_list)

In [ ]:
Y_t = y_true_list
Y_p = y_pred_list

## Confusion Matrix

Creiamo un dataframe dalla matrice di confusione e lo tracciamo come una mappa di calore utilizzando la libreria Seaborn.

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list, normalize='true')).rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
class2idx = {
    "No_action": 0,
    "Action": 1
}

idx2class = {v: k for k, v in class2idx.items()}

In [ ]:
real2class = {
    0:0,
    1:1,
    2:1,
    3:1
}

In [ ]:
y_pred_list = [(real2class[c]) for c in Y_p]
y_true_list = [(real2class[c]) for c in Y_t]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list, normalize='true')).rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
class2idx = {
    "No_action": 0,
    "P/R": 1,
    "Premi": 2
}

idx2class = {v: k for k, v in class2idx.items()}

In [ ]:
real2class = {
    0:0,
    1:1,
    2:1,
    3:2
}

In [ ]:
y_pred_list = [(real2class[c]) for c in Y_p]
y_true_list = [(real2class[c]) for c in Y_t]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list, normalize='true')).rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
class2idx = {
    "No_action": 0,
    "P/P": 1,
    "Rilascia": 2
}

idx2class = {v: k for k, v in class2idx.items()}

In [ ]:
real2class = {
    0:0,
    1:1,
    2:2,
    3:1
}

In [ ]:
y_pred_list = [(real2class[c]) for c in Y_p]
y_true_list = [(real2class[c]) for c in Y_t]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list, normalize='true')).rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
class2idx = {
    "No_action": 0,
    "R/P": 1,
    "Prendi": 2
}

idx2class = {v: k for k, v in class2idx.items()}

In [ ]:
real2class = {
    0:0,
    1:2,
    2:1,
    3:1
}

In [ ]:
y_pred_list = [(real2class[c]) for c in Y_p]
y_true_list = [(real2class[c]) for c in Y_t]

In [ ]:
confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true_list, y_pred_list, normalize='true')).rename(columns=idx2class, index=idx2class)
sns.heatmap(confusion_matrix_df, annot=True)

In [ ]:
print(classification_report(y_true_list, y_pred_list))